# Designing hardware experiments with qSDK

An important aspect of the qSDK package is to enable users to design and run succesful quantum chemistry experiments on quantum hardware. By leveraging the various toolboxes in the qSDK, including pre- and post-processing tools, we can take a molecular system to a quantum computer, gauge the feasibility of our approaches on existing hardware, and compare them to alternatives.

The goal of this notebook is to show how the building blocks provided can support our exploration while leveraging systematic approaches to keep track of resource requirements and anticipate accuracy of results on existing quantum devices.

We here focus on the computation of ground state energy, through the lens of a few variational approaches.

This notebook assumes that you already have installed qSDK in your Python environment, or have updated your PYTHONPATH accordingly.

Subjects
* DMET-QCC
* McWeeny purification
* Grouping term

## Table of contents:
* [1. Choose your own adventure](#1)
* [2. Developing systematic approaches](#2)
* [3. Problem Decompositon](#3)
* [4. Exploration with variational algorithms](#4)
* [5. Making the most of our measurements](#5)
* [6. Circuit optimisation](#6)
* [7. Submitting experiments to quantum devices](#7)
* [8. Post-processing of results](#8)
* [Closing words](#99)

## 1. Choose your own adventure <a class="anchor" id="1"></a>

We start by creating a few molecules with `pyscf`, to support the examples in this notebook. You can easily switch them in most cells below, to observe the impact of the various features on these different problem instances.

In [36]:
import numpy as np
                 
def generate_ring(n, dist, atom="H"):
    xyz = [(atom, (0., dist))]
    
    for i in range(n-1):
        # Rotation of the vector connecting the previous point.
        x = xyz[i][1][0]*np.cos(2*np.pi/n) - xyz[i][1][1]*np.sin(2*np.pi/n)
        y = xyz[i][1][0]*np.sin(2*np.pi/n) + xyz[i][1][1]*np.cos(2*np.pi/n)
        
        # Rounding with 3-decimals precision.
        x = round(x, 3)
        y = round(y, 3)
        
        xyz.append((atom, (x, y)))

    return xyz

In [1]:
from qsdk import SecondQuantizedMolecule
from copy import deepcopy


xyz_H2 = [('H', (0, 0, 0)),('H', (0, 0, 0.74137727))]
xyz_H4 = [('H', ( 0.7071067811865476, 0, 0)), ('H', (0,  0.7071067811865476, 0)),
          ('H', (-1.0071067811865476, 0, 0)), ('H', (0, -1.0071067811865476, 0))]
xyz_H8 = [('H', (0, 0, 0)), ('H', (0, 0, 0.9)), ('H', (0, 0.9, 0)), ('H', (0, 0.9, 0.9)),
          ('H', (0.9, 0, 0)), ('H', (0.9, 0, 0.9)), ('H', (0.9, 0.9, 0)), ('H', (0.9, 0.9, 0.9))]
xyz_H2O = [('O', (0, 0, 0.1173)),('H', ( 0, 0.7572, -0.4692)), ('H', ( 0, -0.7572, -0.4692))]

mol_H2 = SecondQuantizedMolecule(xyz_H2, q=0, spin=0, basis="sto-3g", frozen_orbitals=None)
mol_H4 = SecondQuantizedMolecule(xyz_H4, q=2, spin=0, basis="minao", frozen_orbitals=None)
mol_H8 = SecondQuantizedMolecule(xyz_H8, q=0, spin=0, basis="sto-3g", frozen_orbitals=None)
mol_H2O = SecondQuantizedMolecule(xyz_H2O, q=0, spin=0, basis="sto-3g", frozen_orbitals=None)

<br>

## 2. Developing systematic approaches <a class="anchor" id="2"></a>

Can quantum computers be used to compute something meaningful with regards to a molecular system ? How can we assess the feasibility of an approach, or compare it to others ?

Before delving into the features this package offers, which can quickly turn into a rabbit hole of parameters and heuristics, we emphasize the need to leverage different tools and metrics in order to guide our exploration.

### Resource estimation

Resource estimation helps us assert the feasibility of an approach with regards to device capabilities (simulator or QPU), or compare it to alternatives, including what is considered state-of-the-art.

Resource estimation is particularly relevant now as quantum computing is still a nascent field, and the current quantum computers have modest capabilities (limited amount of qubits and coherence time, low gate fidelity...). It can be one of the drivers of our exploration, and identify both the most appropriate approaches in our experiments and their bottlenecks, where impactful breakthroughs would make a difference.

### Classical simulators

Quantum computers currently have limited access and capability. To study the applications of quantum computing on small problem instances, we can use classical simulators and emulators in order to anticipate the behavior of quantum algorithms on real device, in the presence or absence of noise. 

This package provides support for a collection of open-source quantum circuit simulators, delivering different performance and features. We are free to choose the most relevant backend for our usecases, thinking about resource requirements, use of shots, presence or absence of noise, and accuracy of simulation, for example.

### Comparing accuracy with classical solvers

As meeting the required accuracy for practical applications is essential, wrappers to several standard classical solvers are provided for convenience in this package, such as FCI or CCSD. They are not intended to be competitive solutions, but simply to support us with asserting the accuracy of our quantum explorations, on the systems they are able to handle.

<br>

## 3. Problem decomposition <a class="anchor" id="3"></a>

Problem Decomposition (PD) techniques may be able to help us tackle a molecular system that would be too challenging to tackle head-on with quantum algorithms. We have explored methods such as DMET in the past in our own hardware experiments, and showed how it may reduce resource requirements while preserving accuracy for some systems such as a ring of 10 hydrogen atoms, or several hydrocarbons. If you are interested, we encourage you to have a look at the dedicated DMET notebook, which delves a bit more into the theory and some of the results obtained.

This package also offers other PD techniques, such as ONIOM, to look at systems

In the example below, we show how DMET can reduce resource requirements for a simple system, and splitting it into 2 subproblems. In the rest of the notebook, we look at examples not involving problem decomposition, in order to keep the code clear and simple: it is however possible to combine problem decomposition to the other features and tools we introduce below

In [2]:
#import py3Dmol
#view = py3Dmol.view(width=200,height=200,viewergrid=(1,1))
#H8_py3dmol = open('h8_cube.xyz', 'r').read()
#view.addModel(H8_py3dmol,'xyz',viewer=(0,0))
#view.setStyle({'stick':{'colorscheme':'cyanCarbon'}})
#view.zoomTo()
#view.show()

In [2]:
from qsdk.problem_decomposition.dmet.dmet_problem_decomposition import DMETProblemDecomposition
from qsdk.problem_decomposition.dmet.dmet_problem_decomposition import Localization
from qsdk.electronic_structure_solvers import VQESolver, FCISolver, CCSDSolver
from qsdk.electronic_structure_solvers.vqe_solver import BuiltInAnsatze

mol = mol_H8

# Resource estimation of head-on VQE approach on our system
vqe_options = {"molecule": mol, "ansatz": BuiltInAnsatze.UCCSD, "qubit_mapping": "jw", "verbose": False}
vqe_solver = VQESolver(vqe_options)
vqe_solver.build()
print(f"VQE-UCCSD JW resource estimation \n{vqe_solver.get_resources()}\n")

# Same VQE approach, but performing PD first with DMET (4 fragments)
dmet_options = {"molecule": mol, "verbose": False,
                "fragment_atoms": [2]*4, "fragment_solvers": ["vqe"] + ["ccsd"]*3}
dmet_solver = DMETProblemDecomposition(dmet_options)
dmet_solver.build()
print(f"DMET-VQE-UCCSD resource-estimation, 4 fragments \n{dmet_solver.get_resources()[0]}\n")


# Same VQE approach, but performing PD first with DMET (8 fragments)
dmet_options = {"molecule": mol, "verbose": False,
                "fragment_atoms": [1]*8, "fragment_solvers": ["vqe"] + ["ccsd"]*7}
dmet_solver = DMETProblemDecomposition(dmet_options)
dmet_solver.build()
print(f"DMET-VQE-UCCSD resource-estimation, 8 fragments \n{dmet_solver.get_resources()[0]}\n")


# Same VQE approach, but performing PD first with DMET (8 fragments) + sbBK
dmet_options = {"molecule": mol, "verbose": False,
                "fragment_atoms": [1]*8, "fragment_solvers": ["vqe"] + ["ccsd"]*7,
                "solvers_options": [{"qubit_mapping": "scBK", "initial_var_params": "ones", "up_then_down": True, "verbose": False}] + [{}]*7,
               "verbose": False}
dmet_solver = DMETProblemDecomposition(dmet_options)
dmet_solver.build()
print(f"DMET-VQE-UCCSD resource-estimation, 8 fragments, scBK mapping \n{dmet_solver.get_resources()[0]}\n")

VQE-UCCSD JW resource estimation 
{'qubit_hamiltonian_terms': 2385, 'circuit_width': 16, 'circuit_gates': 29576, 'circuit_2qubit_gates': 17728, 'circuit_var_gates': 1344, 'vqe_variational_parameters': 152}

DMET-VQE-UCCSD resource-estimation, 4 fragments 
{'qubit_hamiltonian_terms': 145, 'circuit_width': 8, 'circuit_gates': 2388, 'circuit_2qubit_gates': 1152, 'circuit_var_gates': 144, 'vqe_variational_parameters': 14}

DMET-VQE-UCCSD resource-estimation, 8 fragments 
{'qubit_hamiltonian_terms': 27, 'circuit_width': 4, 'circuit_gates': 158, 'circuit_2qubit_gates': 64, 'circuit_var_gates': 12, 'vqe_variational_parameters': 2}

DMET-VQE-UCCSD resource-estimation, 8 fragments, scBK mapping 
{'qubit_hamiltonian_terms': 9, 'circuit_width': 2, 'circuit_gates': 20, 'circuit_2qubit_gates': 4, 'circuit_var_gates': 4, 'vqe_variational_parameters': 2}



Let's run the `simulate` method to find the ground state energy of this molecule using the latter approach:

In [3]:
dmet_energy = dmet_solver.simulate()

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8436821206660374
            Iterations: 6
            Function evaluations: 24
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8437821845423783
            Iterations: 6
            Function evaluations: 24
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.842814578070494
            Iterations: 6
            Function evaluations: 24
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8428925476978748
            Iterations: 6
            Function evaluations: 24
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8428997568858523
            Iterations: 6
            Function evaluati

In [4]:
print(f"\n DMET-VQE-UCCSD optimized single-fragment \n {dmet_solver.get_resources()[0]}\n")
print(f"CCSD energy:\t\t\t{CCSDSolver(mol).simulate():.5f}")
print(f"DMET-VQE 8 fragments results:\t{dmet_energy.real:.5f}")


 DMET-VQE-UCCSD optimized single-fragment 
 {'qubit_hamiltonian_terms': 9, 'circuit_width': 2, 'circuit_gates': 20, 'circuit_2qubit_gates': 4, 'circuit_var_gates': 4, 'vqe_variational_parameters': 2}

CCSD energy:			-3.87752
DMET-VQE 8 fragments results:	-3.87742


As we can see, problem decomposition techniques such as DMET can have an important impact on resource requirements, and can contribute to making problems more tractable for simulators and current quantum devices.

## 4. Exploration with variational algorithms <a class="anchor" id="4"></a>

Variational algorithms such as VQE are quantum-classical approaches that have received a lot of attention from the community in the recent years, yielding shallow circuits that are closer to the capabilities of current quantum hardware, and thus may provide opportunities for meaningful hardware experiments now. 

This is why we implemented a number of them, and tried to provide a flexible interface allowing us to try different options: our implementation of VQE got a dedicated notebook, if you are interested.

Variational algorithms can however be a bit tricky, as they require a classical optimization problem to be solved with heuristics: it is hard to quantify the accuracy, and overall runtime or resources required (such as the number of measurements) to run it using a quantum device. Moreover, the quantum circuit and qubit Hamiltonian may change during the optimisation process, depending on your flavour of VQE. This complexity can be used to illustrate the usefulness of the resource estimation, classical simulators and classical solvers mentioned in the previous section, and develop a sense of the challenges in improving existing quantum algorithms or designing succesful hardware experiments.

### 4.1 Before diving into quantum <a class="anchor" id="41"></a>

Even for modest problem instances, "cheap" insights in our molecular systems may prove to considerably lower resource requirements or improve accuracy in our calculations. This can make the difference between being able to explore a particular use case and running a succesful hardware experiment, or not. 

There's no rule that says we must not help quantum computers with our classical tools and chemist intuition, or what is relevant to attain chemical accuracy (~ 1kcal/mol), although this may render some approaches questionable for all practical applications on devices mature enough. Maybe in the future, some of these classical insights could be obtained at low cost (possibly through advances in machine learning, for example) and play well with our quantum workflows. Who knows.

We provide a few simple tools for convenience, but we leave it to the user to gain these insights through other computational or experimental means.

### 4.2 Quantum exploration <a class="anchor" id="42"></a>

The following section has a closer look at H2O, a "small" use case that experts have been trying to tackle with quantum computers in the last few years, big enough to still pose a challenge in minimal basis set.

Using chemist insights, we know that freezing the core should have negligible impact on the accuracy of ground state energy calculation: we first confirm this using our CCSD solver. In fact, our implementation of VQE freezes the core orbitals by default, and leaves it to the user to override this behaviour if they'd like to explore something different.

The cell below shows that for this particular use case, freezing the core orbitals allows us to remove 2 qubits, and reduces the size of the qubit Hamiltonian by half. The number of VQE variational parameters was reduced by 30%, which may help VQE converge faster, too. The number of gates in our variational ansatz circuit -here UCCSD- depends on the value of variational parameters (our implementation only simulates the gates that are necessary), but is likely to remain shallower once parameters have been optimized too.

In [5]:
from qsdk.electronic_structure_solvers import FCISolver, CCSDSolver
from qsdk.electronic_structure_solvers.vqe_solver import BuiltInAnsatze, VQESolver
from qsdk.toolboxes.ansatz_generator.uccsd import UCCSD

mol = mol_H2O

# Impact of frozen core on ground state energy calculation
print(f"Exact FCI energy:\t\t\t {FCISolver(mol).simulate()}")
print(f"CCSD Energy without freezing core:\t {CCSDSolver(mol).simulate()}")
print(f"CCSD Energy with frozen core:     \t {CCSDSolver(mol.freeze_mos('frozen_core', inplace=False)).simulate()}")

# VQE: no frozen orbitals
vqe_options = {"molecule": mol, "ansatz": BuiltInAnsatze.UCCSD, "qubit_mapping": 'jw'}
vqe_solver = VQESolver(vqe_options)
vqe_solver.build()
print(f"\nVQE-UCCSD JW without frozen core \n {vqe_solver.get_resources()}\n")

# VQE default: frozen core
vqe_options = {"molecule": mol.freeze_mos('frozen_core', inplace=False), "ansatz": BuiltInAnsatze.UCCSD, "qubit_mapping": 'jw'}
vqe_solver = VQESolver(vqe_options)
vqe_solver.build()
print(f"\nVQE-UCCSD JW with frozen core \n {vqe_solver.get_resources()}\n")

# Save qubit Hamiltonian for later cells
qb_ham_H2O = deepcopy(vqe_solver.qubit_hamiltonian)

Exact FCI energy:			 -75.01257824109095
CCSD Energy without freezing core:	 -75.0124617015436
CCSD Energy with frozen core:     	 -75.01238336243863

VQE-UCCSD JW without frozen core 
 {'qubit_hamiltonian_terms': 1086, 'circuit_width': 14, 'circuit_gates': 6914, 'circuit_2qubit_gates': 3824, 'circuit_var_gates': 360, 'vqe_variational_parameters': 65}


VQE-UCCSD JW with frozen core 
 {'qubit_hamiltonian_terms': 551, 'circuit_width': 12, 'circuit_gates': 4008, 'circuit_2qubit_gates': 2112, 'circuit_var_gates': 224, 'vqe_variational_parameters': 44}



It's challenging for classical computers to simulate quantum circuits: most exact simulators require an amount of resources that grows exponentially with number of qubits. Running the above VQE on H2O with a good simulator may take a little while: let's switch to our smaller H4 molecule to look at the circuits VQE returns

In [6]:
mol = mol_H4

vqe_options = {"molecule": mol, "ansatz": BuiltInAnsatze.UCCSD, "qubit_mapping": 'jw'}
vqe_solver = VQESolver(vqe_options)
vqe_solver.build()
print(f"Resource requirements (initial parameters) \n{vqe_solver.get_resources()}\n")
vqe_energy = vqe_solver.simulate()
print(f"\nResource requirements (optimized parameters) \n{vqe_solver.get_resources()}\n")

# Compare to FCI (exact) energy
print(f"Difference with FCI energy : {abs(FCISolver(mol).simulate() - vqe_energy):.3E}")

Resource requirements (initial parameters) 
{'qubit_hamiltonian_terms': 185, 'circuit_width': 8, 'circuit_gates': 790, 'circuit_2qubit_gates': 368, 'circuit_var_gates': 52, 'vqe_variational_parameters': 9}

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.8546073434094339
            Iterations: 5
            Function evaluations: 54
            Gradient evaluations: 5

Resource requirements (optimized parameters) 
{'qubit_hamiltonian_terms': 185, 'circuit_width': 8, 'circuit_gates': 1398, 'circuit_2qubit_gates': 688, 'circuit_var_gates': 84, 'vqe_variational_parameters': 9}

Difference with FCI energy : 1.429E-06


We could also try to use the Hardware-Efficient ansatz (HEA), which is a great example of ansatz that may benefit greatly from the use of penalty terms:

In [7]:
initial_var_params =  [ 6.15, 6.19, 6.23, -3.03, -2.20, -7.19, -3.07, 3.41, -0.35, 3.19, -5.58, -7.94,
                       -0.13, 1.51, 1.67, 4.81, -4.81, 2.45, 0.27, 1.59, -5.23, -1.80, -3.75,  0.09]
hea_penalty = {"N": [10,2], "Sz": [10,0], "S^2": [0,0]}
hea_options = {"n_layers": 3, "rot_type": "real", "reference_state": 'zero'}

vqe_options = {"molecule": mol_H4.freeze_mos([3], inplace=False), "qubit_mapping":"bk",
               "ansatz": BuiltInAnsatze.HEA, "initial_var_params" : initial_var_params,
               "penalty_terms": hea_penalty,"ansatz_options": hea_options}

vqe_solver_h4_frozen = VQESolver(vqe_options)
vqe_solver_h4_frozen.build()
print(f"\nDifference with FCI energy : {abs(vqe_solver_h4_frozen.simulate() - vqe_energy):.3E}\n")
print(f"Resources: {vqe_solver_h4_frozen.get_resources()}")

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.850972442919653
            Iterations: 33
            Function evaluations: 854
            Gradient evaluations: 33

Difference with FCI energy : 3.635E-03

Resources: {'qubit_hamiltonian_terms': 62, 'circuit_width': 6, 'circuit_gates': 39, 'circuit_2qubit_gates': 15, 'circuit_var_gates': 24, 'vqe_variational_parameters': 24}


### 4.3 So many possibilities <a class="anchor" id="43"></a>

Variational algorithms present many parameters and options, and it would not be possible to explore all of them. It can already be difficult to gauge whether classical optimization in a particular case did not work out, or the ansatz performed poorly. Speaking of which: you can provide your very own variational ansatz to VQE, which we elaborate on in a different notebook.

Whether or not you'd like to try the Hardware-Efficient Ansatz (HEA), a flavor of Qubit Coupled-Cluster (QCC), or take a stab at the problem with ADAPT-VQE is up to you.

## 5. Making the most of our measurements<a class="anchor" id="5"></a>

It is important to gather as many measurements of the quantum state prepared by our quantum circuits, in order to extract accurate information, used to estimate properties such as energies. In particular, the ground state energy of a system can be computed as an expectation value of a quantum state w.r.t its qubit Hamiltonian, in the variational approaches we've seen so far.

However, gathering more measurements requires a more extensive usage of quantum computers, increasing runtime and therefore the cost of the experiment. It is important to try to make the most of our measurements, to help keep both cost and runtime moderate, while providing satifying accuracy. 

Several techniques of various complexity can be employed to estimate the number of measurements required to meet a given accuracy, or reduce the amount of measurements required to compute expectation values: we'll briefly discuss a few.

For the sake of this example, let's have a look at H2O again. During our own exploration, we came across the following circuit, using ADAPT-VQE. The resulting energy was less than 3 mHa away from FCI, which was not bad for such a shallow circuit (see below). In comparison, running VQE-UCCSD got us to chemical accuracy, but took longer to simulate and resulted in a final circuit of about 20000 gates.

In [8]:
with open("adapt_final_circuit_H2O.qasm", "r") as f:
    adapt_H2O_circuit_qasm = f.read()

from agnostic_simulator.translator import _translate_openqasm2abs
adapt_H2O_circuit_abs = _translate_openqasm2abs(adapt_H2O_circuit_qasm)
print(f"H2O ADAPT-VQE circuit: Gate count = {adapt_H2O_circuit_abs.size}\n {adapt_H2O_circuit_abs.counts}")

from agnostic_simulator import Simulator
sim = Simulator(target="qulacs")
print(f"Energy: {sim.get_expectation_value(qb_ham_H2O, adapt_H2O_circuit_abs)}")

FileNotFoundError: [Errno 2] No such file or directory: 'adapt_final_circuit_H2O.qasm'

### 5.1 Discarding negligible qubit Hamiltonian terms <a class="anchor" id="51"></a>

There are several methods that exist to identify terms that do not significantly contribute to the expectation value of an operator or Hamiltonian, from simple heuristics and manual exploration tailored to the use case, to more systematical and robust approaches. 

A few convenience functions here can help us identify what terms are potentially negligible, or what is the impact of discarding these terms on the computation of the energy. In the future, we plan to support more robust and systematical approaches.

In [ ]:
qb_ham = qb_ham_H2O

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from agnostic_simulator.helpers.operators import print_histogram_coeffs

print_histogram_coeffs(qb_ham)

In [ ]:
expv_exact = sim.get_expectation_value(qb_ham_H2O, adapt_H2O_circuit_abs)
print(f"Exact value: {expv_exact} \t\t({len(qb_ham_H2O.terms)} terms)")

qb_ham = deepcopy(qb_ham_H2O)
qb_ham.compress(abs_tol=5e-3)
expv = sim.get_expectation_value(qb_ham, adapt_H2O_circuit_abs)
print(f"Difference after compressing: {expv - expv_exact:.3E} \t({len(qb_ham.terms)} terms)")

### 5.2 Grouping Hamiltonian terms <a class="anchor" id="52"></a>

It is possible to derive the expectation values of several qubit Hamiltonian terms from a circuit measured in a single measurement basis. By identifying groups of terms whose expectation value can be computed from a single histogram of measurements, we may considerably reduce the amount of circuit executions and measurements required to evaluate expectation values of qubit operators.

In particular, we can group Hamiltonian terms using qubitwise commutativity. The algorithm returns a data-structure whose number of entries is the number of measurement bases required (and thus the number of "quantum tasks" or "jobs") to compute the estimation value of our qubit Hamiltonian.

In [ ]:
from agnostic_simulator.helpers import qubitwise_commutativity_of
res = qubitwise_commutativity_of(qb_ham_H2O, seed=0)
print(f"Using QWC, we may only require to execute {len(res)} circuits, instead of {len(qb_ham_H2O.terms)}.")

There exists more elaborate techniques to form larger groups of terms, that may be supported in the future in this package.

### 5.3 Estimating the number of measurements <a class="anchor" id="53"></a>

Estimating the number of measurements required to evaluate the expectation value of an operator up to a certain accuracy is an active topic of research. There exist simple heuristics that make no assumption about the final quantum state or problem, while some others may be more elaborate and leverage knowledge from classical simulators or require additional quantum gates in our circuits.

The method below implements a simple heuristic making no assumption on the quantum state resulting from our circuit, implementing a simple paradigm based on binomial distribution.

In [ ]:
from agnostic_simulator.helpers import get_measurement_estimate

# Compute and display the measurement estimates for a few operators
res = get_measurement_estimate(qb_ham_H2O, digits=2, method="uniform")

for k,v in list(res.items())[:10]:
    print(k,v)

## 6. Circuit optimization <a class="anchor" id="6"></a>

Frequently, quantum circuits implementing a particular unitary transformation can be improved by identifying sequences of gates that can be replaced by a more simple one, or even remove gates or qubits that are not necessary to the computation.

When aiming for a hardware experiment, one may want to consider the constraints of physical quantum device such as connectivity (qubit topology) and the native gate set of the device. This is the idea of co-design, and raises the question of qubit placement and optimizing circuits for specific gate sets, in order to obtain a circuit that is more likely to perform well on the target harware.

Our package supports various formats that enable the use of such "circuit compilers", often available through third-party packages coming from the hardware providers themselves, or groups designing tools focused on this purpose, including interal efforts at 1QBit.We can leverage such tools and APIs from hardware providers to look at what the circuit running on the target quantum processor may look like.

## 7. Submitting experiments to quantum devices <a class="anchor" id="7"></a>

We support a number of formats representing quantum circuits, which allows us to support various backends, such as simulators or quantum devices. In the algorithms shown above, users could specify `backend_options` as a dictionary, and name their target of choice (qiskit, qulacs, ...): translation into the correct format would be done automatically at runtime.

Users can also use these format convertion functions explicitly. Below, we show how easy it is to obtain a circuit in one of the supported formats, such as IBM Qiskit's. 

In [ ]:
from agnostic_simulator import Gate, Circuit
abs_circuit = Circuit([Gate("H", 0), Gate("CNOT", target=1, control=0)])

from agnostic_simulator import translate_qiskit
qiskit_circuit = translate_qiskit(abs_circuit)
print(qiskit_circuit)

Assuming an account has been set up to use such quantum cloud services, once the circuit is in the adequate format users can make simple calls to the provider's API or our wrappers to submit their experiment, and retrieve the corresponding histograms of measurements.

The cell below shows how to obtain a circuit in the Braket format, and submits it to the local Braket simulator for simplicity (e.g no need for a Braket account in this example): the circuit can be submitted to various backends without changes. For submitting an experiment on a quantum device, see the official Braket API documentation.

In [ ]:
# Translate circuit into Braket format
from agnostic_simulator import translate_braket
braket_circuit = translate_braket(abs_circuit)

# Submit the Braket circuit to a target device using the Braket API
from braket.devices import LocalSimulator as BraketLocalSimulator
device = BraketLocalSimulator()
n_shots = 1000
braket_results = device.run(braket_circuit, shots=n_shots).result()

# Retrieve the histogram of results for post-processing
histogram = braket_results.measurement_counts
print(histogram)

## 8. Post-processing of results <a class="anchor" id="8"></a>

Once a histogram of measurements has been obtained, post-processing can take place. We are able to use the histogram of measurements to compute quantities such as expectation values, or use noise-mitigation techniques to account for the noise in the device, and get results that may be closer to the theoretical ones.

Some noise-mitigation techniques may not require any knowledge of our chemical system, such as the bootstrapping technique or calibration matrices, while some other may leverage information from it, such as Mc Weeny's density matrix purification technique.

Below we show how an expectation value can be computed from a histogram of measurements.

In [ ]:
from agnostic_simulator.helpers import pauli_string_to_of

# Rescale for frequencies
freqs = {k:v/n_shots for k,v in histogram.items()}
print(f"Frequencies: {freqs}\n")

# Compute expectation value for all-Z operator
n_qubits = len(list(freqs.keys())[0])
exp_allZ = Simulator.get_expectation_value_from_frequencies_oneterm(pauli_string_to_of("Z"*n_qubits), freqs)
print(f"Expectation value for all-Z operator {exp_allZ:.5f}")

## Closing words <a class="anchor" id="99"></a>

The goal of this notebook was to show how to explore all the different steps of an end-to-end hardware experiment pipeline using qSDK. We intend to keep adding the best of what we and the rest of the community produce to help everyone designing fruitful quantum hardware experiments.